# Resolución del Examen Data Science - OPI Analytics
## Gonzalo Lima Cinta

***
### Sección B
### Quién es quién en los precios

In [5]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import date, timedelta, datetime
import time

In [39]:
sc = SparkSession.builder.appName("PysparkProfeco").config ("spark.sql.shuffle.partitions", "50")\
.config("spark.driver.maxResultSize","2g").config ("spark.sql.execution.arrow.enabled", "true").getOrCreate()

In [114]:
data = sc.read.option("header","true").csv('databases/all_data.csv')

In [43]:
data.describe().show()

+-------+--------------------+--------------------+--------------------+--------------------+--------+------------------+--------------------+---------------+--------------------+---------------+--------------------+--------------------+--------------+------------------+-------------------+
|summary|                 _c0|                 _c1|                 _c2|                 _c3|     _c4|               _c5|                 _c6|            _c7|                 _c8|            _c9|                _c10|                _c11|          _c12|              _c13|               _c14|
+-------+--------------------+--------------------+--------------------+--------------------+--------+------------------+--------------------+---------------+--------------------+---------------+--------------------+--------------------+--------------+------------------+-------------------+
|  count|            62530716|            62530716|            62530716|            61643378|62530488|          62530716|   

In [115]:
data.show(10)

+--------------------+--------------------+------------------+----------------+----------------+------+--------------------+------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------+----------+
|            producto|        presentacion|             marca|       categoria|        catalogo|precio|       fechaRegistro|   cadenaComercial|                giro|     nombreComercial|           direccion|          estado|           municipio| latitud|  longitud|
+--------------------+--------------------+------------------+----------------+----------------+------+--------------------+------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------+----------+
|CUADERNO FORMA IT...|96 HOJAS PASTA DU...|          ESTRELLA|MATERIAL ESCOLAR|UTILES ESCOLARES|  25.9|2011-05-18 00:00:...|ABASTECEDORA LUMEN|          PAPELERIAS|ABASTECEDORA LUME...|CANNES No. 6 ESQ....

***
### 1.- Procesamiento de datos

***
__a.- ¿Cuántos registros hay?__

Existen 62,530,716 de registros

In [42]:
data.count()

62530716

***
__b.- ¿Cuántas categorías?__

Existen 42 categorías

In [44]:
len(data.select('categoria').distinct().collect())

42

***
__c.- ¿Cuántas cadenas comerciales están siendo monitoreadas?__

Se registran 706 diferentes cadenas comerciales

In [27]:
len(data.select('cadenaComercial').distinct().collect())

706

***
__d.- ¿Cómo podrías determinar la calidad de los datos? ¿Detectaste algún tipo de
inconsistencia o error en la fuente?__

Al realizar una limpieza removiendo duplicados se redujo el total de registros de 62 millones 500 mil (aprox.) a
Además se podrían eliminar registros con algun dato faltante o inconsistente.  (no se hizo el análisis)
La página de donde se descargaron los datos indica que tiene registro de más de 2mil productos, sin embargo se encontraron solamente 1,107. Así mismo, hizo falta un diccionario de datos o tabla explicativa de la base, para un completo entendimiento de cada dato.
Prácticamente todos los registros cuentan con toda la información. La columna con más faltantes es "categoría" con alrededor de 800mil faltantes.
Al realizar una lista de los diferentes estados, se detectaron colonias o calles en dicha columna.

***
__e.- ¿Cuáles son los productos más monitoreados en cada entidad?__

La bae de datos contiene 1107 diferentes productos

In [34]:
len(data.select('producto').distinct().collect())

1107

***
__f.- ¿Cuál es la cadena comercial con mayor variedad de productos monitoreados?__

Soriana es la cadena con mayor variedad con 1059 productos diferentes

In [72]:
cadenas = data.groupBy("cadenaComercial").agg(countDistinct("producto"))

In [76]:
cadenas.orderBy('count(producto)', ascending=False).show(5)

+--------------------+----------+
|                 _c7|count(_c0)|
+--------------------+----------+
|             SORIANA|      1059|
|            WAL-MART|      1051|
|MEGA COMERCIAL ME...|      1049|
|  COMERCIAL MEXICANA|      1036|
|            CHEDRAUI|      1026|
+--------------------+----------+
only showing top 5 rows



### 2.- Análisis exploratorio

***
__a.- Genera una canasta de productos básicos que te permita comparar los precios
geográfica y temporalmente. Justifica tu elección y procedimiento__

Se optó por seleccionar una canasta con los 5 productos con más registros.
Refresco, Detergente para ropa, Fud, Leche Ultrapasteurizada, shampoo

Utilizando los resultados de los incisos siguientes, se logra observar cuales son los estados con mayor precio para esta "canasta básica" así como los precios por mes. Quiero destacar que la ciudad de México no figura entre los 15 estados con mayor costo. Así también se logra observar que el mes de Enero fue en promedio el más caro de todo el año.

In [98]:
productos = data.groupBy("producto").count()

In [155]:
productos.orderBy('count', ascending=False).show(10,False)

+-----------------------+-------+
|_c0                    |count  |
+-----------------------+-------+
|REFRESCO               |1247981|
|DETERGENTE P/ROPA      |990122 |
|FUD                    |933410 |
|LECHE ULTRAPASTEURIZADA|886716 |
|SHAMPOO                |745467 |
|JABON DE TOCADOR       |744914 |
|CHILES EN LATA         |724862 |
|MAYONESA               |697586 |
|YOGHURT                |632362 |
|DESODORANTE            |623684 |
+-----------------------+-------+
only showing top 10 rows



***
__b. ¿Cuál es la ciudad más cara del país? ¿Cuál es la más barata?__

No se cuenta con el dato de ciudades, pero si de municipios, por lo que se presenta el resultado en base a dicho nivel. Se seleccionaron los 5 productos más comunes y se hizo el promedio de esos productos en cada municipio.

MONCLOVA fue el municipio más cara con un promedio del precio en los productos de $45 mientras que TLAJOMULCO DE ZUÑIGA fue el municipio con los productos más económicos con un promedio del precio de $12.8

In [124]:
top_prods = data.filter(data.producto.isin("REFRESCO","DETERGENTE P/ROPA","FUD","LECHE ULTRAPASTEURIZADA","SHAMPOO"))
prod_mun = top_prods.groupBy("municipio").agg({'precio':'avg'})

In [125]:
#promedio más alto
prod_mun.orderBy('avg(precio)', ascending=False).show(5,False)

+----------------------------------------+------------------+
|municipio                               |avg(precio)       |
+----------------------------------------+------------------+
|MONCLOVA                                |45.06075514874144 |
|CUAUTITLÁN                              |44.91681918359528 |
|SAN LUIS POTOSÍ                         |44.86105761389828 |
|CARMEN                                  |44.41721173658613 |
|REYNOSA                                 |44.206107197549706|
+----------------------------------------+------------------+
only showing top 5 rows



In [ ]:
#promedio más bajo
prod_mun.orderBy('avg(precio)', ascending=True).show(5,False)

***
__c. ¿Hay algún patrón estacional entre años?__

No parece haber un patrón estacional. En 2011 los meses con el mayor precio fueron noviembre y diciembre, en 2012 fueron febrero y marzo, en 2013 septiembre y octubre ocuparon el mayor promedio. 
El unico patrón facilmente detectable es el aumento gradual año tras año del precio de estos 5 productos.

In [117]:
#Extracción de año y mes a partir de fecha
split_date=split(data['fechaRegistro'], '-')
data = data.withColumn('Year', split_date.getItem(0))
data = data.withColumn('Month', split_date.getItem(1))

In [133]:
prod_year = top_prods.groupBy(["year","month"]).agg({'precio':'avg'})

In [138]:
prod_year.orderBy('year', ascending=False).show(72,False)

+----+-----+------------------+
|year|month|avg(precio)       |
+----+-----+------------------+
|2016|04   |40.780153894440915|
|2016|03   |40.61842780366333 |
|2016|02   |40.47524414739435 |
|2016|01   |40.793362613423575|
|2015|08   |39.475693356105445|
|2015|10   |40.70794611361077 |
|2015|04   |38.27207695827156 |
|2015|09   |40.541086343893355|
|2015|01   |37.94318128233905 |
|2015|12   |41.53846067079426 |
|2015|05   |38.75451954816243 |
|2015|07   |39.084128241205086|
|2015|06   |39.02409806217784 |
|2015|02   |38.32796863822286 |
|2015|03   |38.26804693040797 |
|2015|11   |41.07190792561249 |
|2014|11   |37.586010191988   |
|2014|03   |35.15733521755276 |
|2014|01   |35.27421479109234 |
|2014|12   |37.346299448475605|
|2014|09   |37.89643961256737 |
|2014|04   |36.03578641713857 |
|2014|06   |37.090682472855434|
|2014|05   |36.04165324583158 |
|2014|10   |37.66927767299489 |
|2014|08   |36.99401233762523 |
|2014|02   |34.85938319349745 |
|2014|07   |37.09593012994573 |
|2013|01

***
__d. ¿Cuál es el estado más caro y en qué mes?__

San Luis Potosí es el estado más caro y el mes de Diciembre tuvo el promedio mas alto seguido de Abril.

In [126]:
prod_ent = top_prods.groupBy(["estado","month"]).agg({'precio':'avg'})

In [132]:
prod_ent.orderBy('avg(precio)', ascending=False).show(15,False)

+---------------+-----+------------------+
|estado         |month|avg(precio)       |
+---------------+-----+------------------+
|SAN LUIS POTOSÍ|12   |45.32688813349815 |
|SAN LUIS POTOSÍ|04   |45.22754942058623 |
|SAN LUIS POTOSÍ|01   |44.97118375325803 |
|SAN LUIS POTOSÍ|05   |44.80866485013625 |
|SAN LUIS POTOSÍ|03   |44.771965703397896|
|SAN LUIS POTOSÍ|11   |44.636701244813274|
|SAN LUIS POTOSÍ|09   |44.54791610648023 |
|SAN LUIS POTOSÍ|02   |44.22057390189163 |
|SAN LUIS POTOSÍ|06   |44.219844647078695|
|SAN LUIS POTOSÍ|07   |43.7720568181818  |
|SAN LUIS POTOSÍ|10   |43.66876453864048 |
|SAN LUIS POTOSÍ|08   |43.341147836538475|
|CAMPECHE       |09   |43.25322056574924 |
|CAMPECHE       |12   |43.18041896719714 |
|GUERRERO       |01   |43.1501078546307  |
+---------------+-----+------------------+
only showing top 15 rows



*Para complementar la respuesta al inciso a

In [158]:
prod_ent = top_prods.groupBy(["estado"]).agg({'precio':'avg'})
prod_ent.orderBy('avg(precio)', ascending=False).show(15,False)

+--------------------+------------------+
|estado              |avg(precio)       |
+--------------------+------------------+
|SAN LUIS POTOSÍ     |44.43373228593054 |
|CAMPECHE            |42.30598875215309 |
|GUERRERO            |41.82067367204908 |
|TAMAULIPAS          |41.24869568335763 |
|BAJA CALIFORNIA SUR |40.632374062950355|
|NAYARIT             |40.527602769951415|
|CHIHUAHUA           |40.52042394125954 |
|COAHUILA DE ZARAGOZA|40.1405524096436  |
|AGUASCALIENTES      |39.89019901470129 |
|QUINTANA ROO        |39.41741205293447 |
|MORELOS             |39.38802589810472 |
|DURANGO             |39.02842752659573 |
|BAJA CALIFORNIA     |37.99886646047012 |
|CHIAPAS             |37.63936392268267 |
|MÉXICO              |36.42039169052019 |
+--------------------+------------------+
only showing top 15 rows



In [159]:
prod_ent = top_prods.groupBy(["month"]).agg({'precio':'avg'})
prod_ent.orderBy('avg(precio)', ascending=False).show(15,False)

+-----+------------------+
|month|avg(precio)       |
+-----+------------------+
|01   |36.564854590506386|
|09   |36.53325812792576 |
|12   |36.38284965520774 |
|10   |36.21843633645343 |
|11   |35.92251238484839 |
|04   |35.897136101899555|
|03   |35.79061615419941 |
|02   |35.69699568829376 |
|08   |35.62398818100341 |
|07   |35.61535501390084 |
|06   |35.16025094563259 |
|05   |34.432575079659344|
+-----+------------------+



***
__e. ¿Cuáles son los principales riesgos de hacer análisis de series de tiempo con
estos datos?__

La inconsistencia en los datos es el primer factor de riezgo. Cada estado vende productos diferentes y en diferentes cantidades. Es por esto que se seleccionaron solamente los 5 productos más vendidos para hacer los análisis anteriores. Además pudieron haberse ido introduciendo nuevos productos año con año, por lo que haría falta un análisis más exaustivo.

***
### 3.- Visualización

__a.- Genera un mapa que nos permita identificar la oferta de categorías en la zona metropolitana de León Guanajuato y el nivel de precios en cada una de ellas. Se darán puntos extra si el mapa es interactivo__

Se hará la suposición que el municipio de "LEÓN y LEON" incluyen la zona metropolitana.

In [153]:
data.select('estado').distinct().collect()

[Row(estado='QUINTANA ROO'),
 Row(estado=' ESQ. SUR 125"'),
 Row(estado='NUEVO LEÓN'),
 Row(estado='SINALOA'),
 Row(estado='TABASCO'),
 Row(estado='BAJA CALIFORNIA SUR'),
 Row(estado='JALISCO'),
 Row(estado='CAMPECHE'),
 Row(estado='CHIAPAS'),
 Row(estado=' COL. EDUARDO GUERRA'),
 Row(estado='NAYARIT'),
 Row(estado='BAJA CALIFORNIA'),
 Row(estado='TLAXCALA'),
 Row(estado='CHIHUAHUA'),
 Row(estado='DISTRITO FEDERAL'),
 Row(estado='estado'),
 Row(estado='VERACRUZ DE IGNACIO DE LA LLAVE'),
 Row(estado='HIDALGO'),
 Row(estado='ZACATECAS'),
 Row(estado='COAHUILA DE ZARAGOZA'),
 Row(estado='QUERÉTARO'),
 Row(estado='SONORA'),
 Row(estado='YUCATÁN'),
 Row(estado='GUANAJUATO'),
 Row(estado='MICHOACÁN DE OCAMPO'),
 Row(estado='SAN LUIS POTOSÍ'),
 Row(estado='TAMAULIPAS'),
 Row(estado='DURANGO'),
 Row(estado='MORELOS'),
 Row(estado='GUERRERO'),
 Row(estado=None),
 Row(estado='AGUASCALIENTES'),
 Row(estado='OAXACA'),
 Row(estado='COLIMA'),
 Row(estado='MÉXICO'),
 Row(estado='PUEBLA')]

In [152]:
#Distintos municipios en guanajuato
data.filter(data.estado == 'GUANAJUATO').select('municipio').distinct().collect()

[Row(municipio='LEÓN'),
 Row(municipio='LEON                                    ')]

In [149]:
leon = data.filter(data.municipio.isin('LEON','LEÓN'))

In [150]:
categorias_leon = leon.groupBy("categoria").agg({'precio':'avg'})

In [154]:
categorias_leon.orderBy('avg(precio)', ascending=True).show(10,False)

+-------------------------------+------------------+
|categoria                      |avg(precio)       |
+-------------------------------+------------------+
|TE                             |13.80720073664825 |
|CIGARRILLOS                    |14.961790668348042|
|TORTILLAS Y DERIVADOS DEL MAIZ |15.290977703455969|
|REFRESCOS ENVASADOS            |16.199623414423783|
|PAN                            |17.65706231079718 |
|FRUTAS Y LEGUMBRES PROCESADAS  |19.37406217526842 |
|PESCADOS Y MARISCOS EN CONSERVA|19.881359691417554|
|CHOCOLATES Y GOLOSINAS         |20.356773314650475|
|LEGUMBRES SECAS                |20.99267949226498 |
|HORTALIZAS FRESCAS             |21.59279772284331 |
+-------------------------------+------------------+
only showing top 10 rows



***
## Notas finales
Por el tamaño de la base, fue complicado realizar algunos procesamientos debido a la falta de memoria del dispositivo en el que se trabajó (macbook air - 8gb ram - < 2gb en disco libres despues de descargar la base)

Cabe destacar que nunca había utilizado spark (a pesar de haber estudiado sus capacidades y funcionalidades) y sin embargo, resultó de gran utilidad para el procesamiento de esta base de datos de más de 20GB